In [147]:
import numpy as np
import pandas as pd 
from tqdm import tqdm 
from functools import lru_cache
import os 

In [148]:
recstudio_data_dir = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/data_for_recstudio'
raw_data_dir = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/raw_data'

In [149]:
@lru_cache(maxsize=1)
def read_valid_data():
    return pd.read_csv(os.path.join(recstudio_data_dir, 'all_task_1_valid_sessions.csv'))

@lru_cache(maxsize=1)
def read_product_data():
    return pd.read_csv(os.path.join(raw_data_dir, 'products_train.csv'))

In [150]:
valid_sessions_df = read_valid_data()
valid_sessions_df.head(5), len(valid_sessions_df)

(                                                                                       prev_items   next_item locale
 0  ['B09VSN9GLS' 'B09VSG9DCG' 'B0BJ5L1ZPH' 'B09VSN9GLS' 'B0BJ6V797Y'\n 'B09VSG9DCG' 'B077XGDMD2']  B06XG1LZ6Z     UK
 1                                                        ['B00390YWXE' 'B00390YWXE' 'B09WM9W6WQ']  B01MSUI4FE     JP
 2                                                        ['B01BM9V6H8' 'B01MG55XDR' 'B07VYSSRL7']  B01M6625ME     UK
 3               ['B092ZG24S7' 'B09BNHWWZM' 'B08CB1WG5M' '1788009975' '1839941960'\n '1788009975']  0241558573     UK
 4                                                                     ['B0B6NY5RM8' 'B09BJGBBBR']  B09BJF6N8K     JP,
 361581)

In [151]:
valid_sessions_df = valid_sessions_df[valid_sessions_df['locale'] == 'UK'].reset_index(drop=True)
len(valid_sessions_df)

130665

In [152]:
valid_sessions_df

,prev_items,next_item,locale
0,['B09VSN9GLS' 'B09VSG9DCG' 'B0BJ5L1ZPH' 'B09VSN9GLS' 'B0BJ6V797Y'\n 'B09VSG9DCG' 'B077XGDMD2'],B06XG1LZ6Z,UK
1,['B01BM9V6H8' 'B01MG55XDR' 'B07VYSSRL7'],B01M6625ME,UK
2,['B092ZG24S7' 'B09BNHWWZM' 'B08CB1WG5M' '1788009975' '1839941960'\n '1788009975'],0241558573,UK
3,['B07V5LWSN5' 'B0BF1CGGSJ' 'B07W4DH45P' 'B08P395NWN' 'B08DNF166V'\n 'B07W6JP96X' 'B0919KJY3B' 'B08JYCJ8GC' 'B08CDMG1GK' 'B09NBSLVZS'\n 'B013SL1YXO'],B07W6J5FB2,UK
4,['B07P4CYP4S' 'B07P4CYP4S' 'B0B7XDNP5W'],B0B7XCS3QT,UK
...,...,...,...
130660,['B07GKM97YF' 'B07GKP2LCF'],B07GKYSHB4,UK
130661,['B0B2JCPSFK' 'B0B2HTT64B' 'B0BBWMJJTK'],B08QGSHVKV,UK
130662,['B086YQN87Q' 'B086DP9T8Z'],B09WTGMWXC,UK
130663,['B08Q67YV3S' 'B07Y36TGRS' 'B08Q67YV3S' 'B08BZFVK19'],B09C1YJS7D,UK


In [153]:
session_candidates = [[] for _ in range(len(valid_sessions_df))]

In [154]:
len(session_candidates), len(session_candidates[0])

(130665, 0)

In [155]:
with open('./valid_results/valid_UK_ranking.txt', 'r') as f:
    for line in f:
        session_list = line.split('\t')
        session_id = session_list[0]
        product = session_list[1]
        session_candidates[int(session_id)].append(product.split('_')[0])


In [156]:
len(session_candidates), len(session_candidates[0])


(130665, 350)

In [1]:
# history is filtered
for i in tqdm(range(len(session_candidates))):
    new_candidates = []
    hist_candidates = eval(valid_sessions_df.iloc[i]['prev_items'].replace(' ', ','))
    for product in session_candidates[i]:
        if product not in hist_candidates:
            new_candidates.append(product)
    session_candidates[i] = new_candidates[:300]

NameError: name 'tqdm' is not defined

In [158]:
valid_predict_df = pd.DataFrame({'locale' : valid_sessions_df['locale'], 'next_item_prediction' : session_candidates}).reset_index(drop=True)
valid_predict_df.head(5)

,locale,next_item_prediction
0,UK,"[B06XGD9VLV, B06XG1LZ6Z, B06XGDZVZR, B0BJQ95XWV, B09WVLM2TB, B09XDVQFWD, B0BJ66G5LN, B0B2N3RGMF, B09BD65VSF, B09NPZPKYQ, B09NQ1GK4P, B08BNH48JM, B09DXD4STW, B087FB54VN, B087DYVDXL, B087F749NB, B098QH6122, B09P54SPTQ, B09P52T43Z, B095NZWZDH, B07FCBV2Q5, B077VSMJ8R, B0B2JT2BND, B08F7P8R6D, B0B2JSKN7X, B01HF2A8PC, B097DM36WD, B0B9BKBCLC, B084ZQ5JF4, B09TP6PWNB, B09SLBY68P, B077VXHCCH, B07BK7HMN2, B09TW5134P, B085Y1L7LX, B085Y1DP3P, B07G5ZSNY2, B08VN7FMLT, B09WM8JX9T, B07BK7739T, B07BK7HZ2L, B09XQSP9LD, B09T6QTGC8, B097Y7S2KY, B097Y7MDHN, B084ZQ2NW8, B09W1V6Q3M, B098QF9QPP, B00I0UKKD4, B09TZS4S2Z, B07TH98T68, B07TGBMC72, B084ZPN91X, B092ZQDZF5, B07TBQB8FK, B09TWBBFY9, B09FY916F1, B077VXNL6H, B083QXSY66, B084QF8P74, B079BL4CPZ, B01MPYILA7, B01MYUDYP7, B07SNHRTSW, B07WQ4B3WZ, B097PVPKV4, B09FY7VX9S, B09YXB4JKW, B077VX8Q12, B082L9D9R4, B09SPZ3RT1, B076PN1SKG, B084ZPL21L, B077VXNL67, B077VWJYHT, B07RNZZ89K, B07RMJ5X1R, B07RK95WGT, B07T8FX6L2, B072HCH1R2, B0728GGX6Y, B071XXBXDT, B01MZXG8J0,..."
1,UK,"[B00JLH125A, B09GGGFKFR, B015DVC6G6, B017A0OY86, B00KP0UETW, B06XX9W93F, B00L17OXRM, B0B6CTFYX6, B01GJ6ICYO, B07VS2H3P6, B01M8KBTGW, B09X5XLZR9, B0BGSX7DL5, B07BMG1S8X, B01IT3KVB2, B09XBQ8XSK, B01M8I1WX4, B01MG1CO07, B01LPOSCBI, B01CDUR2S2, B07GJS6QNP, B01MDOBUCC, B079WKK1B2, B0BG647MF9, B0B1D4665T, B07D3GG47V, B0B6CW5HCN, B09GB1312D, B09QJ2N5C2, B0744HJQK9, B091G673KC, B01F02DYLY, B01MQ62UJV, B081NBB59N, B00HP48OCU, B096YFM2K5, B01MAYMOCG, B01CNSNSPA, B01CCZ4LNM, B09XBQ5DSZ, B091G35Q5T, B01LF3WD6O, B091G67DJ9, B00M4GYASG, B01LPOS1W8, B01M7ULTTN, B07115S814, B09TRRX7XT, B01N20N3FB, B0BB3F54F5, B0B1QK35XL, B094DMFLC6, B07GJS6QPF, B00CD31U24, B08SK6SPCP, B00FQV015C, B01JZTY29Q, B08DJR5NWN, B09P1PFV3H, B01N2G0TLF, B01IT3KKNQ, B01CJSI624, B01MU1PF62, B08F2Q6FRX, B00H0GFY3K, B01GJ6ICVC, B01AWH5AW8, B08FMKZFV2, B073PGNDTP, B0B6CW77DS, B01EYKV5U0, B01GRWA4F0, B01AWH4JFC, B01IB2SSA2, B01IT3KURW, B0BF62Q3Z7, B01BNLTTRE, B01NCB79N3, B0BC9SBPN8, B00JLH126Y, B07CT2JP2L, B075VVPVSL, B01CNSNSHI,..."
2,UK,"[1406357383, 1406363073, B0BGY899Q5, B074G22VZC, 0008544840, 0008305749, B01M5EBAO4, B07G4MNXPZ, B0BGF31237, 0439873770, B00E32X422, 0552545104, B09ZTG7344, 1405296895, B07XYYX7ZM, B09GXHNH7F, 1801313652, B0BJ54PZV6, B095LR178T, 140528272X, B0154LBMFG, B08D6DD4NG, B0BC1FTLZS, 1405290528, B0B7L5LPLW, 0593125177, 024135739X, 000847902X, 1408354403, 1841215651, 184869069X, B09XB5G823, B0B6F3FGQZ, B09ZTGMS34, B09ZSMGFMT, B098NHBNR6, B07W5V4Q6B, B097H5X5M1, B0B8DWYPK6, 0803738412, 1509804773, B00IMKXIBI, 150983043X, B07X9LJST9, 1529025346, 0241456541, B07HMVLQB2, 0141376147, 0141376139, B009S8AZ64, B08DP195R4, B095YMFCGR, 0241357349, 0241357306, B097JZ86B5, 1787009726, 1803700890, B08CL1L5M3, B097JK5BSK, 1407170740, B09BL1NLS1, B09734YB3P, B07X95M3RV, B0BBXLD735, B08FJ29PYV, B09C1WPN4K, B094R357SS, B09F3Q6MNZ, B09BQT82M3, 1801314918, 152906421X, B0B778ZK32, B08HD2BDLP, B07GY2JB9H, B08BVXRTMM, B094QTHLMH, B09DN1FKN1, B09BVWGLDY, 0008560749, B09BVWTKM6, B087MVYZ3C, B0B1ZM95GZ, B0933FSJRZ,..."
3,UK,"[B013SL2712, B07W6JP976, B07W6JJ253, B07W7LHVBM, B07W4DHYR9, B07W7L11LJ, B013SL2050, B01GV6HP0E, B013SL26LS, B07W4DHYLC, B002MMY4WY, B004N627KS, B0BF1D5W4D, B00A389JEW, B08FBZW5QD, B08FBZ8QSL, B08FBZVDM5, B00XJ3LVPI, B0957ZDFJ2, B0074486GA, B0055JDM06, B07K2PK3BV, B08HKYNY73, B0B8C4Z38P, B09NXMPRGG, B016NY7784, B07W7K1LRV, B095BXKSFP, B09G344JRY, B07W4DG8RR, B00OK49SMI, B09JKDYY3B, B085RGMMV4, B07TH9L7QR, B07W6HPP3R, B08J7ZP1J3, B07W5JK5RN, B07W7K8MW2, B07V3HG2YD, B0B97XCNL2, B0B925S1HT, B09JLRPVP7, B07JJYLKQ2, B003ZY9Z40, B0B599ZN13, B096XW7WBL, B08T5QJCTM, B07W6JLQCY, B07Y3T2XXV, B0B6PQKG78, B09HTW1QMJ, B0B8SRBV4R, B08W55GSTV, B0B599KRXJ, B081Z3VZS5, B019O1R4DQ, B07XRFMG5B, B01EHSLFAY, B08MLBPNQY, B08CXF9WF5, B085WYXXLF, B07W4DH1Y5, B07W7JLVCL, B07W6J5FB2, B08CH44W4S, B08BCJNMGP, B09WYJQG64, B09MFL38PC, B0B3926J39, B00CL6353A, B00X5P80IQ, B07XTGS1J3, B0B8NKMT5S, B085XGLXRJ, B07XYM62G6, B01MT5WLHY, B0919LWWS3, B0B58RQZZM, B0BB1YR

In [159]:
def cal_hit_and_mrr(ground_truth_list, candidates_list):
    hits, mrrs = [], []
    for i in tqdm(range(len(ground_truth_list))):
        ground_truth = ground_truth_list.iloc[i]
        candidates = candidates_list.iloc[i]
        hit, mrr = 0.0, 0.0
        for j in range(len(candidates)):
            if ground_truth == candidates[j]:
                hit = 1.0
                mrr = 1.0 / (j + 1)
                break
        hits.append(hit)
        mrrs.append(mrr)
    return np.array(hits).mean(), np.array(mrrs).mean()

In [160]:
cal_hit_and_mrr(valid_sessions_df['next_item'], valid_predict_df['next_item_prediction'])

100%|██████████| 130665/130665 [00:02<00:00, 45920.32it/s]


(0.7314047373053227, 0.24190486057082744)

In [170]:
cal_hit_and_mrr(valid_sessions_df['next_item'], valid_predict_df['next_item_prediction'].apply(lambda x : x[:100]))

100%|██████████| 130665/130665 [00:02<00:00, 57812.77it/s]


(0.6577201239811732, 0.24143956782486917)

In [169]:
valid_predict_df.to_parquet('./valid_results/valid_prediction.parquet', engine='pyarrow')

In [146]:
product_data = read_product_data()
product_data = product_data.set_index('id')

In [171]:
def compare_title(index):
    print('********last item*********')
    prev_items = eval(valid_sessions_df.iloc[index]['prev_items'].replace(' ', ','))
    last_item = prev_items[-1]
    last_item_title = product_data.loc[last_item][['title']]
    print(last_item_title)
    print('*******ground truth*******')
    next_item = valid_sessions_df.iloc[index]['next_item']
    next_item_title = product_data.loc[next_item][['title']]
    print(next_item_title)
    print('*******prediction*********')
    for i in range(5):
        print(f'No. {i} : ')
        candidate = valid_predict_df.iloc[index]['next_item_prediction'][i]
        predict_title = product_data.loc[candidate][['title']]
        print(predict_title)

In [172]:
compare_title(200)

********last item*********
title    Police Hat Outfit Accessory for Police Cops Fancy Dress
Name: B0057WXSNC, dtype: object
*******ground truth*******
title    HAT POLICEMAN ADULT (beanie/hat)
Name: B005CC679E, dtype: object
*******prediction*********
No. 0 : 
title    ADULTS POLICE HAT STAG NIGHT HAT - POLICE OFFICER HAT POLICEMAN'S CAP FOR FANCY DRESS - ONE SIZE FITS MOST PANDA HAT
Name: B00I4YCLIS, dtype: object
No. 1 : 
title    WPC Hat Outfit Accessory for Police Cops Fancy Dress
Name: B0054YL9I4, dtype: object
No. 2 : 
title    Smiffys Police Panda Cap Fabric Check Band - Black
Name: B000LWCN98, dtype: object
No. 3 : 
title    1X POLICE PANDA CAP FANCY DRESS ACCESSORY CAP HAT WITH CHECKED BAND & BADGE BULK COP HATS
Name: B071NNN1QM, dtype: object
No. 4 : 
title    Lizzy Adult Policeman Hat Police Cop Fancy Dress Costume Peak Cap With Check Band and Handcuffs (Unisex Police Hat & Metal handcuff)
Name: B06XQZ2X1T, dtype: object


In [95]:
compare_title(100)


********last item*********
title    edding 22 EcoLine permanent marker set - black, red, blue, green - 4 pens - chisel nib 1-5 mm - waterproof, quick-drying, smear-proof pens - for cardboard, plastic, glass, wood, metal and fabric
Name: B001E39KGW, dtype: object
*******ground truth*******
title    STAEDTLER 341 WP6 Lumocolor whiteboard compact markers, pack of 6 assorted colours
Name: B07GS12BSQ, dtype: object
*******prediction*********
No. 0 : 
                                                                                                                                                                                                title
id                                                                                                                                                                                                   
B00473V0N2  edding 21 EcoLine permanent marker - black - 1 pen - round nib 1.5-3 mm - waterproof, quick-drying, smear-proof pens - for cardboard, plastic,

In [68]:
pd.set_option('max_colwidth',1000)

In [82]:
valid_sessions_df.iloc[1000]

prev_items    ['B0132XVLDG' 'B0132XVLDG' 'B00NFR1M7U' 'B00NFR1M7U' 'B09L1LZ991']
next_item                                                             B09L1LZ991
locale                                                                        UK
Name: 1000, dtype: object

In [97]:
compare_title(1000)

********last item*********
                                                                                                               title
id                                                                                                                  
B09L1LZ991  Eveready Gold Batterien AA, 40 Stück, Langlebige Batterien für Haushaltsgeräte Alkaline, Amazon-exklusiv
B09L1LZ991                                           Eveready Gold Alkaline AA Batteries, 40 Pack [Amazon Exclusive]
*******ground truth*******
                                                                                                               title
id                                                                                                                  
B09L1LZ991  Eveready Gold Batterien AA, 40 Stück, Langlebige Batterien für Haushaltsgeräte Alkaline, Amazon-exklusiv
B09L1LZ991                                           Eveready Gold Alkaline AA Batteries, 40 Pack [Amazon Exclusive]
*******pre

In [98]:
compare_title(2000)

********last item*********
                                                                                                                                                        title
id                                                                                                                                                           
B085WV4WST                                                LEGO 21166 Minecraft Die verlassene Mine Bauset, Zombiehöhle mit Figuren: Schleim, Steve und Spinne
B085WV4WST                                                                                   レゴ(LEGO) マインクラフト 廃坑の探検 21166 おもちゃ ブロック プレゼント テレビゲーム 男の子 女の子 7歳以上
B085WV4WST      LEGO 21166 Minecraft The Abandoned Mine Building Toy, Zombie Cave with Slime, Steve & Spider Figures, Gift idea for Boys and Girls Age 7 plus
B085WV4WST  LEGO 21166 Minecraft La Mina Abandonada, Juguete de Construcción para Niños de 7 Años o Más, Juego de Acción con Ladrillos, Araña, Zombie y Slime
B085WV4WST   LEGO 21166 M

In [99]:
compare_title(5601)

********last item*********
title    Deadly Cry: An absolutely gripping crime thriller packed with suspense
Name: B08FJ71854, dtype: object
*******ground truth*******
title    Twisted Lies: An absolutely gripping mystery and suspense thriller
Name: B08W2CDXVR, dtype: object
*******prediction*********
No. 0 : 
title    Deadly Secrets: An absolutely gripping crime thriller
Name: B07956JX2T, dtype: object
No. 1 : 
title    Lethal Game: An absolutely gripping crime thriller packed with suspense
Name: B08YY13YTV, dtype: object
No. 2 : 
title    Silent Voices: A gripping crime thriller packed with mystery and suspense
Name: B08KT53KGV, dtype: object
No. 3 : 
title    Little Girls Sleeping: An absolutely gripping crime thriller
Name: B07PM81LJZ, dtype: object
No. 4 : 
title    Lola Is Missing: A totally gripping crime thriller
Name: B077JH259R, dtype: object


In [100]:
compare_title(45677)

********last item*********
title    G9 LED Bulbs 6-Pack, TopLeder 5W 450LM LED Light Bulbs, Warm White 3000K, No Flicker Non-Dimmable, 40W Halogen Bulb Equivalent, Energy Saving G9 Bulb
Name: B09GY5BJ7N, dtype: object
*******ground truth*******
title    G9 LED Bulbs Cool White 5W, TopLeder G9 Bulbs 450LM 6500K Equivalent to 40W 50W Halogen Bulb, Led G9 Bulbs, G9 Socket Led Lamp, No Flicker Non-Dimmable, AC 220-240V, 6 Pack
Name: B08SHX4ZWV, dtype: object
*******prediction*********
No. 0 : 
title    G9 LED Bulbs Cool White 5W, TopLeder G9 Bulbs 450LM 6500K Equivalent to 40W 50W Halogen Bulb, Led G9 Bulbs, G9 Socket Led Lamp, No Flicker Non-Dimmable, AC 220-240V, 6 Pack
Name: B08SHX4ZWV, dtype: object
No. 1 : 
title    G9 Led Bulbs, 5W Warm White 3000K,40W Halogen Light Bulbs Equivalent,470Lm, Non Dimmable,Pack of 4
Name: B09TH74WWF, dtype: object
No. 2 : 
title    G9 LED Light Bulbs, 5W, Warm White 3000K, 450LM, Equivalent to 40W G9 Halogen Bulb, Non Dimmable, AC 220V-240V for Lighting 

In [101]:
compare_title(45647)

********last item*********
title    RelianceUK 30 Pack 5 x 5 x 5” Single Wall Cardboard Boxes Small Postal Box for Mailing Shipping Packaging
Name: B0B71X39FG, dtype: object
*******ground truth*******
title    RedOne Hair Styling Aqua Wax Red 150 ml | Edge Control | Hair Gel Wax | Ultra Hold | Strawberry Scent | Men & Women Hair Wax | Maximum Control
Name: B07XVP3R4F, dtype: object
*******prediction*********
No. 0 : 
title    RelianceUK 25 Pack 8 x 6 x 4” Single Wall Cardboard Boxes Small Postal Box for Mailing Shipping Packaging
Name: B0B71WWVHB, dtype: object
No. 1 : 
title    RelianceUK 20 Pack 6 x 6 x 6” Single Wall Cardboard Boxes Small Postal Box for Mailing Shipping Packaging
Name: B0B71VMF3Q, dtype: object
No. 2 : 
title    RelianceUK 20 Pack 9 x 6 x 6” Single Wall Cardboard Boxes Small Postal Box for Mailing Shipping Packaging
Name: B0B71XPZ9P, dtype: object
No. 3 : 
title    50 x Single Wall 12" x 9" x 6" Inch (30.5 x 22.9 x 15 cm) Cardboard Small Box, Royal Mail Small Parcel

In [102]:
compare_title(65647)

********last item*********
title    Frame's by Lewis's - Cologne Black Portrait Frame - (30cm x 42cm) A3 Photo Frame, Picture Display Frame, Mantlepieces, Wall Mount
Name: B095HK134C, dtype: object
*******ground truth*******
title    Hampton Frames BLACKWOOD A3 (30x42cm) Black Painted Pine Wood Photo Picture Poster Frame BLW3042BLKX
Name: B07YZH9V8H, dtype: object
*******prediction*********
No. 0 : 
title    MH Traders Black A3 Picture Frame - Wall Mounted Photo Frame for Pictures Posters & Certificates - 30 x 42 CM A3 Size Compact Picture Frames
Name: B09PJH17DS, dtype: object
No. 1 : 
title    MH Traders Silver A3 Picture Frame - Wall Mounted Photo Frame for Pictures Posters & Certificates - 30 x 42 CM A3 Size Compact Picture Frames
Name: B0B1QGX126, dtype: object
No. 2 : 
title    Hampton Frames CAMBRIDGE A3 (30x42cm) Black Picture Poster Photo Frame Plexi (Non-Glass) CAM3042NG
Name: B093CBD3NJ, dtype: object
No. 3 : 
title    Hampton Frames BACKLOADER A3 (30x42cm) Black Picture Pos